# 说明函数

In [1]:
def say_hello():
    print("Hello12!")

greeting = say_hello
greeting()  # 输出: Hello!

# 函数可以作为参数传递给其他函数
def execute(func):
    func()

execute(say_hello)  # 输出: Hello!

# 函数可以作为返回值
def get_greeting():
    return say_hello

greeting_func = get_greeting()
greeting_func()  # 输出: Hello!


# map()是一个高阶函数，接受一个函数和一个可迭代对象
numbers = [1, 2, 3, 4, 5]
squared = map(lambda x: x**2, numbers)
print(list(squared))  # 输出: [1, 4, 9, 16, 25]


def outer_function(x):
    def inner_function(y):
        return x + y  # 内部函数可以访问外部函数的变量x
    return inner_function

closure = outer_function(10)
print(closure(5))  # 输出: 15

Hello12!
Hello12!
Hello12!
[1, 4, 9, 16, 25]
15


另一个理解装饰器的关键概念是闭包（Closure）。闭包是指在一个函数内部定义的函数，并且该内部函数可以访问外部函数的变量，即使外部函数已经执行完毕

In [4]:
def outer_function(x):
    def inner_function(y):
        return x + y  # 内部函数可以访问外部函数的变量x
    return inner_function

closure = outer_function(10)
print(closure(5))  # 输出: 15

15


# 装饰器

装饰器的基本结构通常包含以下几个部分：
装饰器函数：
- 接受一个函数作为参数
- 包装函数（Wrapper）：定义在装饰器函数内部，用于增强原始函数的功能
- 返回包装函数：替换原始函数的调用逻辑

In [ ]:
def decorator_function(original_function):
    def wrapper_function():
        # 在原始函数调用前执行的代码
        result = original_function()  # 调用原始函数
        # 在原始函数调用后执行的代码
        return result  # 返回原始函数的结果
    return wrapper_function

有两种使用方式，一种是使用注解

In [5]:
def decorator(func):
    def wrapper():
        print("Before function call")
        func()
        print("After function call")
    return wrapper

@decorator  # 应用装饰器
def say_hello():
    print("Hello, world!")

say_hello()  # 调用装饰后的函数

Before function call
Hello, world!
After function call


一种是直接调用

In [ ]:
def decorator(func):
    def wrapper():
        print("Before function call")
        func()
        print("After function call")
    return wrapper

def say_hello():
    print("Hello, world!")

# 直接应用装饰器
say_hello = decorator(say_hello)

say_hello()  # 调用装饰后的函数

前面的例子展示了如何装饰不带参数的函数。当装饰器应用于带参数的函数时，需要确保包装函数能够正确接收和传递参数。
这可以通过在包装函数中使用*args和**kwargs来实现，这两个特殊参数可以捕获任意位置参数和关键字参数。

In [8]:
def decorator(func):
    def wrapper(*args, **kwargs):
        print(f"调用函数: {func.__name__}")
        print(f"参数: args={args}, kwargs={kwargs}")
        result = func(*args, **kwargs)  # 传递参数给原始函数
        print(f"返回值: {result}")
        return result  # 返回原始函数的结果
    return wrapper

@decorator
def add(a, b):
    return a + b

@decorator
def greet(name, greeting="Hello"):
    return f"{greeting}, {name}!"

print(add(3, 5))
print(greet("Alice", greeting="Hi"))

调用函数: add
参数: args=(3, 5), kwargs={}
返回值: 8
8
调用函数: greet
参数: args=('Alice',), kwargs={'greeting': 'Hi'}
返回值: Hi, Alice!
Hi, Alice!


使用装饰器时需要注意一个重要问题：装饰后的函数会丢失原始函数的元数据（如函数名、文档字符串、参数信息等）

In [9]:
def decorator(func):
    def wrapper():
        print("Before function call")
        func()
        print("After function call")
    return wrapper

@decorator
def say_hello():
    """This function says hello."""
    print("Hello, world!")

print(say_hello.__name__)  # 输出: wrapper，而不是say_hello
print(say_hello.__doc__)   # 输出: None，而不是"This function says hello."

wrapper
None


为了解决这个问题，Python 提供了functools.wraps装饰器，它可以将原始函数的元数据复制到包装函数中

In [10]:
import functools

def decorator(func):
    @functools.wraps(func)  # 关键！保留原始函数的元数据
    def wrapper():
        """包装函数的文档字符串"""
        print("Before function call")
        func()
        print("After function call")
    return wrapper

@decorator
def say_hello():
    """This function says hello."""
    print("Hello, world!")

print(say_hello.__name__)  # 输出: say_hello
print(say_hello.__doc__)   # 输出: This function says hello.
print(say_hello.__module__)# 输出: __main__

say_hello
This function says hello.
__main__


到目前为止，我们看到的装饰器本身都不带参数。但有时候，我们希望装饰器能够接受参数，以实现更灵活的配置。例如，一个可以自定义日志级别的日志装饰器。
带参数的装饰器需要使用三层嵌套结构：
- 最外层函数：接收装饰器参数
- 中间层函数：接收原始函数
- 最内层包装函数：增强原始函数的功能

In [11]:
def decorator_with_args(arg1, arg2):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            # 可以使用arg1和arg2
            result = func(*args, **kwargs)
            return result
        return wrapper
    return decorator

In [15]:
import functools

def log_with_level(level="INFO"):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print(f"[{level}] - 开始执行 {func.__name__}...")
            result = func(*args, **kwargs)
            print(f"[{level}] - {func.__name__} 执行完毕。")
            return result
        return wrapper
    return decorator

@log_with_level(level="DEBUG")  # 传递装饰器参数
def process_data():
    print("Processing data...")

process_data()

[DEBUG22] - 开始执行 process_data...
Processing data...
[DEBUG22] - process_data 执行完毕。


五、类装饰器
除了函数装饰器外，Python 还支持使用类作为装饰器。类装饰器提供了更多的灵活性和状态管理能力，特别适合需要维护复杂状态的场景​
。
5.1 类装饰器的基本结构
类装饰器需要实现__init__和__call__方法：
__init__方法：初始化装饰器实例，接收被装饰的函数作为参数
__call__方法：实现装饰逻辑，类似于函数装饰器中的包装函数

In [ ]:
class DecoratorClass:
    def __init__(self, func):
        self.func = func  # 保存原始函数
        functools.wraps(func)(self)  # 保留原始函数的元数据

    def __call__(self, *args, **kwargs):
        # 在原始函数调用前执行的代码
        result = self.func(*args, **kwargs)  # 调用原始函数
        # 在原始函数调用后执行的代码
        return result  # 返回原始函数的结果

In [19]:
# 以下是一个使用类装饰器实现的日志记录器

import functools

class LogDecorator:
    def __init__(self, func):
        self.func = func
        functools.wraps(func)(self)  # 保留原始函数的元数据

    def __call__(self, *args, **kwargs):
        print(f"调用函数: {self.func.__name__},args={args}, kwargs={kwargs}")
        result = self.func(*args, **kwargs)
        print(f"返回值: {result}")
        return result

@LogDecorator  # 应用类装饰器
def add(a, b):
    return a + b

print(add(3, 5))

调用函数: add,args=(3, 5), kwargs={}
返回值: 8
8


类装饰器也可以接受参数，这需要在__init__方法中处理参数，并在__call__方法中实现装饰逻辑

In [18]:
import functools

class RepeatDecorator:
    def __init__(self, num_times):
        self.num_times = num_times  # 保存装饰器参数

    def __call__(self, func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            results = []
            for a in range(self.num_times):
                results.append(func(*args, **kwargs))
            return results
        return wrapper

@RepeatDecorator(num_times=3)  # 应用带参数的类装饰器
def greet(name):
    print(f"Hello, {name}!")
    return f"Greeted {name}"

results = greet("Alice")
print(results)

Hello, Alice!
Hello, Alice!
Hello, Alice!
['Greeted Alice', 'Greeted Alice', 'Greeted Alice']


装饰器在模块加载时执行，而不是在函数调用时执行。这意味着装饰器代码在模块导入时就会被执行，而装饰后的函数替换过程也在此时完成

In [ ]:
# 模块开始加载时
print("模块开始加载")

def decorator(func):
    print("装饰器被定义")
    @functools.wraps(func)
    def wrapper():
        print("包装函数被调用")
        func()
    return wrapper

@decorator  # 模块加载时，装饰器立即执行，替换say_hello函数
def say_hello():
    print("Hello!")

print("模块加载完成")

say_hello()  # 模块加载完成后调用

In [ ]:
@decorator
def say_hello():
    pass

say_hello.__wrapped__()  # 直接调用原始函数，绕过装饰器